In [3]:
from typing import Any, Dict, Optional
from rich.console import Console
from rich.markdown import Markdown
from dataclasses import dataclass
import os
console = Console()
print = lambda c: console.print(Markdown(c))

from dnd_agent.multiagent import AgentType, RulesAgent, SpatialAgent, NarrativeAgent, CombatAgent, AgentRequest, AgentResponse, MemoryAgent, MultiAgentDnDSystem

from dnd_agent.database.neo4j_manager import Neo4jSpatialManager
from dnd_agent.database.vector_db import PostgresVectorManager

# Set up environment variables
os.environ.setdefault("NEO4J_URI", "bolt://localhost:7687")
os.environ.setdefault("NEO4J_USER", "neo4j")
os.environ.setdefault("NEO4J_PASSWORD", "password")

# Create Neo4j connection
from dnd_agent.database.neo4j_manager import Neo4jSpatialManager
neo4j_manager = Neo4jSpatialManager(
    uri=os.environ["NEO4J_URI"],
    user=os.environ["NEO4J_USER"],
    password=os.environ["NEO4J_PASSWORD"]
)

# Create PostgreSQL connection (optional)
postgres_conn = None
try:
    import psycopg2
    postgres_conn = psycopg2.connect(
        host=os.environ.get("POSTGRES_HOST", "localhost"),
        port=os.environ.get("POSTGRES_PORT", "5432"),
        database=os.environ.get("POSTGRES_DB", "dnd_campaign"),
        user=os.environ.get("POSTGRES_USER", "postgres"),
        password=os.environ.get("POSTGRES_PASSWORD", "password")
    )
except:
    print("PostgreSQL not available - memory features disabled")

@dataclass
class CampaignDeps:
    """Dependencies for the D&D campaign agents"""
    neo4j_driver: Any
    postgres_conn: Optional[Any] = None
    dnd_api_base: str = "https://www.dnd5eapi.co"
    current_context: Optional[Dict[str, Any]] = None

In [1]:
from dataclasses import dataclass
from typing import Any, Dict, Optional

In [2]:
@dataclass
class CampaignDeps:
    """Dependencies for the D&D campaign agents"""
    neo4j_driver: Any
    postgres_conn: Optional[Any] = None
    dnd_api_base: str = "https://www.dnd5eapi.co"
    current_context: Optional[Dict[str, Any]] = None

In [3]:
from dnd_agent.tools.dnd_resources import lookup_dnd_resource

In [4]:
class CTX:
    def __init__(self, deps):
        self.deps = deps

In [5]:
deps = CampaignDeps(
        neo4j_driver=None,
        postgres_conn=None,
        dnd_api_base="https://www.dnd5eapi.co",
        current_context={}
    )


In [6]:
await lookup_dnd_resource(ctx=CTX(deps), resource_type="spells", resource_index="fireball")

'{\n  "index": "fireball",\n  "name": "Fireball",\n  "desc": [\n    "A bright streak flashes from your pointing finger to a point you choose within range and then blossoms with a low roar into an explosion of flame. Each creature in a 20-foot-radius sphere centered on that point must make a dexterity saving throw. A target takes 8d6 fire damage on a failed save, or half as much damage on a successful one.",\n    "The fire spreads around corners. It ignites flammable objects in the area that aren\'t being worn or carried."\n  ],\n  "higher_level": [\n    "When you cast this spell using a spell slot of 4th level or higher, the damage increases by 1d6 for each slot level above 3rd."\n  ],\n  "range": "150 feet",\n  "components": [\n    "V",\n    "S",\n    "M"\n  ],\n  "material": "A tiny ball of bat guano and sulfur.",\n  "ritual": false,\n  "duration": "Instantaneous",\n  "concentration": false,\n  "casting_time": "1 action",\n  "level": 3,\n  "damage": {\n    "damage_type": {\n      "in

In [ ]:
lookup_dnd_resource(
    "spells/fireball", deps.dnd_api_base)

In [6]:
multi_agent_system = MultiAgentDnDSystem(deps, model_config={"model": "gpt-4", "temperature": 0.7})

INFO:dnd_agent.multiagent:Initialized 7 agents


In [10]:
dir(multi_agent_system)[-10:]

['_initialize_agents',
 'agents',
 'deps',
 'direct_agent_call',
 'get_system_status',
 'model_config',
 'process_player_turn',
 'process_user_input',
 'request_log',
 'start_combat']

In [11]:
await multi_agent_system.process(AgentRequest(
    agent_type=AgentType.RULES,
    action="Can I dehydrate a creature using destroy water?",
    context={},
    parameters={}
))

AttributeError: 'MultiAgentDnDSystem' object has no attribute 'process'

In [3]:
agent = RulesAgent('rules', deps, model_config={'model_name': 'gpt-4o'})

In [23]:
result = await agent.agent.run("Can I dehydrate a creature using destroy water?")

In [ ]:
result = await agent.process(AgentRequest(
    agent_type=AgentType.RULES,
    action="Can I dehydrate a creature using destroy water?",
    context={},
    parameters={}
))

In [10]:
print(result.message)

The spell "Create or Destroy Water" allows you to destroy up to 10 gallons of water in an open container within    
range, or alternatively, to destroy fog in a 30-foot cube. The spell description specifies its effects on          
containers and fog but does not extend to dehydrating creatures directly.                                          

Therefore, you cannot use "Destroy Water" to dehydrate a creature based on the spell's description. The spell only 
targets water in an open container or fog, not the water contained within a living being. For the specific         
mechanics and limitations, consult with your Dungeon Master, as they may allow for creative uses at their          
discretion.

In [12]:
AgentType['ENTITY']

<AgentType.ENTITY: 'entity'>

In [13]:
from dnd_agent.tools import recall_chat_history

In [15]:
import psycopg2

postgres_conn = psycopg2.connect(
    host=os.environ["POSTGRES_HOST"],
    port=os.environ["POSTGRES_PORT"],
    database=os.environ["POSTGRES_DB"],
    user=os.environ["POSTGRES_USER"],
    password=os.environ["POSTGRES_PASSWORD"],
)

In [19]:
@dataclass
class Duck:
    deps: Deps
    
@dataclass
class Deps:
    postgres_conn: psycopg2.extensions.connection

In [36]:
memory_agent = MemoryAgent('memory', deps, model_config={'model_name': 'gpt-4o'})

In [37]:
result = await memory_agent.process(AgentRequest(
    agent_type=AgentType.MEMORY,
    action="What chat history do we have?",
    context={},
    parameters={}
))
print(result.message)

It seems there is currently no chat history available for this campaign. If there's anything specific you'd like to
document or discuss for future sessions, feel free to let me know!

In [34]:
spatial_agent = SpatialAgent('spatial', deps, model_config={'model_name': 'gpt-4o'})

In [43]:
result = await spatial_agent.process(AgentRequest(
    agent_type=AgentType.SPATIAL,
    action="Who is located in the rusty dragon inn?",
    context={},
    parameters={}
))
print(result.message)

I currently do not have any information about who is located within the Rusty Dragon Inn. I can set up and track   
the locations of entities if you provide the details or map information of the inn.